Объединенный датасет [FIX MEне доступен по ссылке](*).

Положить в папку data содержимое так, чтобы были следующие пути:  
* \$(ROOT_DIR)/data/full-rtsd/...
* \$(ROOT_DIR)/data/full-gt.csv

> *gt_Set_NaN.csv - содержит тот же датасет, но значения колонки Set обнулено*

gt - датафрейм содержащий:  
* имена файлов - поле filename
* класс знака - поле sign_class
* координаты знаков
* в какой набор включен знак - поле Set $\in$ $\{train, valid, test\}$  

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (25,8)
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL

try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
except:
    IN_COLAB = False

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not IN_COLAB:
    %run utils.ipynb
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

if (NOTEBOOKS_DIR / 'full-gt.csv').is_file():
    full_gt = pd.read_csv(NOTEBOOKS_DIR / 'full-gt.csv')
else:
    full_gt = pd.read_csv(DATA_DIR / 'full-gt.csv')

FORMATED_GT_PATH = "formated_full_gt.csv"
FULL_GT_SRC_LEN = len(full_gt.index)
display(full_gt)

In [ ]:
full_gt_unique_filenames = set(full_gt['filename'])
full_gt_unique_filenames_size = len(full_gt_unique_filenames)
%run utils.ipynb
import ast
import re

i = 0;

if os.path.isfile(FORMATED_GT_PATH):
    print("FORMATED GT EXIST. LOAD IT")
    formated_full_gt_df = pd.read_csv(FORMATED_GT_PATH, dtype=object)
    # display(formated_full_gt_df)
    formated_full_gt_df['coords'].replace({'\n ':',', ' \s+': ' ', '\[ ': '['}, regex=True, inplace=True)
    # display(formated_full_gt_df)
    formated_full_gt_df['coords'] = formated_full_gt_df['coords'].apply(
        lambda x: ast.literal_eval(x)
    )
    
    formated_full_gt_df['size'] = formated_full_gt_df['size'].apply(
        lambda x: ast.literal_eval(x)
    )
else:
    print("FORMATED GT DOESNT EXIST. CREATE IT")
    # get all original filenames
    full_gt_unique_filenames = set(full_gt['filename'])
    # gb = full_gt.groupby('filename', axis=0)
    # gb = gb[['x_from', 'y_from', 'width', 'height']].agg(list)
    # display(gb)
    # create dict
    formated_full_gt_list = []

    import imagesize

    
    for src_filename_iterator in list(full_gt_unique_filenames):

        mask = np.in1d(full_gt['filename'], [src_filename_iterator])
        coord_data_arr = full_gt[mask][['x_from', 'y_from', 'width', 'height']].to_numpy()
        
        filepath = DATA_DIR / "rtsd-frames" / src_filename_iterator
        origW, origH = imagesize.get(filepath)
                
        rel_coord = []
        for coord in coord_data_arr:
            # make from x, y, dx, dx -> x1, y1, x2, y2
            CV2RectangleCoords = ConvertAbsTLWH2CV2Rectangle(coord)

                
            # make from x1, y1, x2, y2 -> x, y, w, h
            CV2CircleCoords = ConvertCV2Rectangle2CenterXYWH(CV2RectangleCoords)
            
            # make x, y, w, h -> relative x, y, w, h
            rel_instance = MakeRel(CV2CircleCoords, origW, origH)

            rel_coord.append(rel_instance)
            
        if i % 100 == 0:
            printProgressEnum(i, full_gt_unique_filenames_size)
        i += 1
        
        
        
        formated_full_gt_list.append([str(filepath), rel_coord, [origW, origH]])
        
    #    formated_full_gt_dict_.append([src_filename_iterator, coord_data])
    #    for coord in coord_data:
    #        origW, origH = imagesize.get(DATA_DIR / "rtsd-frames" / src_filename_iterator)
    #        
    #        relX, relY = (coord[0] + coord[2]) / origW, (coord[1] + coord[3]) / origH 
    #        relW, relH = coord[2] / origW, coord[3] / origH 
    #        
    #        relative_coords = [relX, relY, relW, relH], [origW, origH]

    formated_full_gt_df = pd.DataFrame(formated_full_gt_list, columns=['filepath', 'coords', 'size'])
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)

formated_full_gt_df

# simple test

In [ ]:
instance = formated_full_gt_df.iloc[1]
print(instance)
img = cv2.imread(str(instance['filepath']))
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


h, w = img.shape[0], img.shape[1]
print('Shape:', w, h)


for i in instance['coords']:
    
    xywh = UnmakeRel(i, w, h)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print('+', MakeRel(x1y1x2y2, w, h))
    print('xywh', xywh)
    print('x1y1x2y2', x1y1x2y2)
    
    
    img = cv2.rectangle(img, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)
    
    img = cv2.circle(img, 
                     (xywh[0], xywh[1]), 
                     xywh[2] // 2, 
                     (255, 255, 0), 
                     3)

    
plt.imshow(img)
plt.show()

In [ ]:
if 'set' in formated_full_gt_df.columns:
    print('SET ALREADY EXIST')
else:
    print('SET DOESNT EXIST. LETS CREATE IT')
    formated_full_gt_df_index_count = len(formated_full_gt_df.index)
    TRAIN_SIZE = round(0.7 * formated_full_gt_df_index_count)
    VALID_SIZE = round(0.2 * formated_full_gt_df_index_count)
    TEST_SIZE = round(formated_full_gt_df_index_count - TRAIN_SIZE - VALID_SIZE)
    
    # print('assert:', TRAIN_SIZE + VALID_SIZE + TEST_SIZE, '==', formated_full_gt_df_index_count)
    
    assert TRAIN_SIZE + VALID_SIZE + TEST_SIZE == formated_full_gt_df_index_count, 'wrong split'
    set_series = pd.Series('test', index=range(TEST_SIZE)).append(
        pd.Series('train', index=range(TRAIN_SIZE)).append(
            pd.Series('valid', index=range(VALID_SIZE))
        )
    ).sample(frac=1).reset_index(drop=True)
    formated_full_gt_df['set'] = set_series
    display(formated_full_gt_df)
    formated_full_gt_df.to_csv("formated_full_gt.csv", index=False)
    
display(formated_full_gt_df)

Now we have pd.DataFrame that contains filenames, list of relative coordinates, corresponding photo resoulutions and marks for set. 

Let's make DataSets for it.

In [ ]:
%run utils.ipynb

class createDataSet(torch.utils.data.Dataset):
    def __init__(self, df, set_label, img_size=640, batch_size=16, augment=False, hyp=None, rect=False, image_weights=False,
                 cache_images=False, single_cls=False, stride=32, pad=0.0, prefix=''):
        
        self.img_size = img_size
        self.augment = augment
        self.hyp = hyp
        self.image_weights = image_weights
        self.rect = False if image_weights else rect
        self.mosaic = self.augment and not self.rect  # load 4 images at a time into a mosaic (only during training)
        self.mosaic_border = [-img_size // 2, -img_size // 2]
        self.stride = stride
        self.df = df[df['set']==set_label]
        self.albumentations = Albumentations() if augment else None
        
    def loadImage(self, instance):
        path, (w0, h0) = instance['filepath'], instance['size']
        im = cv2.imread(path)
        assert im is not None, f'Image Not Found {path}'
        
        r = self.img_size / max(h0, w0)  # ratio
        if r != 1:  # if sizes are not equal
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)),
                            interpolation=cv2.INTER_AREA if r < 1 and not self.augment else cv2.INTER_LINEAR)
        return im, (h0, w0), im.shape[:2]
    
    def __getitem__(self, index):
        
        # locate img info from DataFrame
        instance = self.df.iloc[index]
        
        # get Img, src height, width and resized height, width
        img, (h0, w0), (h, w) = self.loadImage(instance)
        
        shape = self.img_size
        
        # make img square
        img, ratio, pad = letterbox(img, shape, auto=False, scaleup=self.augment)
        # print(pad)
        # ?
        shapes = (h0, w0), ((h / h0, w / w0), pad)  # for COCO mAP rescaling
        
        # add class to labels. We have 1 class, so just add zeros into first column
        labels = np.array(instance['coords'])
        labels = np.c_[np.zeros(labels.shape[0]), labels]
        # print(labels)
        
        # fix labels location caused by letterbox
        labels[:, 1:] = xywhn2xyxy(labels[:, 1:], ratio[0] * w, ratio[1] * h, padw=pad[0], padh=pad[1])
        labels[:, 1:5] = xyxy2xywhn(labels[:, 1:5], w=img.shape[1], h=img.shape[0], clip=True, eps=1E-3)

        
        nl = len(labels)  # number of labels
        # if nl:
        #    labels = xyxy2xywhn(labels, w=img.shape[1], h=img.shape[0], clip=True, eps=1E-3)
        
        # why out size (?, 6)?? 
        labels_out = torch.zeros((nl, 6))
        if nl:
            labels_out[:, 1:] = torch.from_numpy(labels)   
        
        # img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
        img = np.ascontiguousarray(img)
        img = torch.Tensor.permute(torch.Tensor(img), [2, 0, 1]).div(255)
        
        return img, labels_out, instance['filepath'], shapes
    
    def __len__(self):
        return len(self.df.index)


IMG_SIZE = 640
train_dataset = createDataSet(formated_full_gt_df,'train', img_size=IMG_SIZE)

img, labels_out, filepath, shapes = train_dataset[1]
print('label_out', labels_out)
print('filepath', filepath)
print('shapes', shapes)

imgNT = cv2.cvtColor(img.permute(1, 2, 0).numpy(), cv2.COLOR_BGR2RGB)

for coord in labels_out[:, 2:]:
    print(coord)
    h, w = shapes[0]
    xywh = UnmakeRel(coord, IMG_SIZE, IMG_SIZE)
    x1y1x2y2 = ConvertCenterXYWH2CV2Rectangle(xywh)
    print(x1y1x2y2)
    imgNT = cv2.rectangle(imgNT, (x1y1x2y2[0], x1y1x2y2[1]), 
                        (x1y1x2y2[2], x1y1x2y2[3]), 
                        (255, 0, 0), 
                        3)
    
plt.imshow(imgNT)
plt.show()

In [ ]:
img, labels_out, filepath, shapes = train_dataset[1]
print(labels_out, filepath, shapes)
cv_img = cv2.cvtColor(img.permute(1, 2, 0).numpy(), cv2.COLOR_BGR2RGB)
print(labels_out[0].numpy())
cv_img = cv2.rectangle(cv_img, )
showImg(cv_img)
shapes


In [ ]:
def createDataLoaderAndDataSet(df, set_label, imgsz, batch_size, stride, hyp=None, augment=False,
                      rect=False, rank=-1, workers=0, image_weights=False, quad=False, prefix='', shuffle=True):
    
    
    
    dataset = CreateDataSet()
    batch_size = min(batch_size, len(dataset))
    
    loader = DataLoader(dataset, # InfiniteDataLoader ?
                        batch_size=batch_size,
                        shuffle=shuffle and sampler is None,
                        # num_workers=nw,  # doesnt work in Windows
                        sampler=sampler,
                        pin_memory=True,
                        collate_fn=LoadImagesAndLabels.collate_fn4 if quad else LoadImagesAndLabels.collate_fn)

    
    return loader, dataset
    
train_dataset = SignDataset(formated_full_gt_df, 'train')
train_dataset[1]

In [ ]:

# valid_dataset = SignDataset(formated_full_gt_df, 'valid')
# test_dataset = SignDataset(formated_full_gt_df, 'test')

In [ ]:
formated_full_gt_df['set'].value_counts()

In [ ]:
formated_full_gt_df['coords'].iloc[-1][0][0]

In [ ]:
import re
# print(formated_full_gt_df['coords'].iloc[-1].replace('\n', ','))
temp1 = formated_full_gt_df['coords'].iloc[-1].replace('\n ', ',').replace('[ ', '[')
print(temp1)
temp = re.sub(' \s+',' ', temp1).replace(' ', ',')
print(temp)
print(ast.literal_eval(temp))

In [ ]:
formated_full_gt_df

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True, classes=1, autoshape=False) #, force_reload=True)

In [ ]:
m = model.model[-1]
# 7,12,  10,17,  13,24,  26,20,  17,32,  23,41,  31,54,  41,72,  58,100 # custom anchors

In [ ]:
m.

In [ ]:
model.train()

In [ ]:
def getSignCoordinates(path=None, filename=None):
    if path == None and filename == None:
        assert 1 != 0, 'cannot identify picture'
    if path:
        filename = str(path).split(sep='\\')[-1]
    located_data = full_gt.loc[full_gt['filename'] == filename]
    return located_data[['x_from', 'y_from', 'width', 'height', 'sign_class']].to_numpy()

import imagesize

def convertFF2YOLO(df, input_dir_abs_path, output_dir_abs_path):
    # print(str(input_dir_abs_path))
    input_files = os.listdir(input_dir_abs_path)
    # print(input_files)
    print(1)
    p = pathlib.Path(output_dir_abs_path)
    p.mkdir(parents=True, exist_ok=True)

    for filename in input_files:
        
        origW, origH = imagesize.get(input_dir_abs_path / filename)
        coord_data = getSignCoordinates(filename=filename)
        print(filename, coord_data)
        if len(coord_data):
            #coord_data = coord_data[0]
            pass
        else:
            # print('Skipped file', filename)
            # continue
            pass
        
        outfilename = output_dir_abs_path / (str(filename).split('.')[-2] + '.txt')
        
        f = open(outfilename, 'wb')
        
        for coord in coord_data:
            # print(coord)
            relX, relY = (coord[0] + coord[2]) / origW, (coord[1] + coord[3]) / origH 
            relW, relH = coord[2] / origW, coord[3] / origH 
            
            str_ = '0 ' + str(relX) + ' ' + str(relY) + ' ' + str(relW) + ' ' + str(relH) + '\n'
            f.write(str_.encode())
        
        f.close()

            
convertFF2YOLO(full_gt, DATA_DIR / 'full-frames/rtsd-frames', DATA_DIR / 'full-frames/rtsd-frames-labels')